<a href="https://colab.research.google.com/github/priyal-shah-portfolio/priyal-shah-portfolio/blob/main/Car_Rental_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datetime import datetime

class Rate:
    def __init__(self, hourly, daliy, weekly):
        self.hourly = hourly
        self.daliy = daliy
        self.weekly = weekly

class Car:
    def __init__(self, rate: Rate, car_plate_number: str):
        self.rate = rate
        self.car_plate_number = car_plate_number
        self.bookings = []

    def get_rate(self):
        return self.rate

    def is_available(self, pickup_time: datetime , drop_off_time: datetime):
        for booking in self.bookings:
            booking_pickup_time = booking.pickup_time
            booking_drop_off_time = booking.drop_off_time
            if pickup_time < booking_drop_off_time and drop_off_time > booking_pickup_time:
                return False
        return True

def get_pick_up_time(customer_license_number: str, car: Car):
    for booking in car.bookings:
        if customer_license_number == booking.customer_license_number:
            return booking.pickup_time
    raise Exception(f"No pick up time found for customer: {customer_license_number} car: {car.car_plate_number}")

class Bill:
    def __init__(self, customer_license_number: str, car: Car, drop_off_time: datetime):
        self.customer_license_number = customer_license_number
        self.car_plate_number= car.car_plate_number
        self.rate = car.get_rate()
        self.drop_off_time = drop_off_time
        self.pickup_time = get_pick_up_time(customer_license_number, car)
        self.duration = self.drop_off_time - self.pickup_time
        total_days = self.duration.days
        week = int(total_days/7)
        days = int(total_days) - (week *7)
        hours = self.duration.seconds//3600
        self.price = self.rate.weekly*(week) + self.rate.daliy*(days) + self.rate.hourly*(hours)


class Booking:
    def __init__(self, pickup_time: datetime, drop_off_time: datetime, customer_license_number: str):
        self.pickup_time = pickup_time
        self.drop_off_time = drop_off_time
        self.customer_license_number = customer_license_number

class Customer:
     def __init__(self, customer_license_number: str):
            self.customer_license_number = customer_license_number

class Inventory:
    def __init__(self, cars: list):
        self.cars = cars

    def get_all_cars(self):
        return self.cars.values()

    def get_car(self, car_plate_number: str):
        if car_plate_number in self.cars.keys():
            return self.cars.get(car_plate_number)
        raise Exception(f"Get_car: No car exists: {car_plate_number}")

    def update_car(self, car: Car):
        if car.car_plate_number in self.cars.keys():
            self.cars[car.car_plate_number] = car
            return
        raise Exception(f"Update_car: No car exists: {car.car_plate_number}")


class BookingCarSystem:
    def __init__(self, inventory: Inventory):
        self.inventory=inventory

    def show_available_cars(self, pickup_time: datetime, drop_off_time: datetime):
        cars = list(self.inventory.get_all_cars())
        available_cars = []
        for car in cars:
            if car.is_available(pickup_time, drop_off_time):
                available_cars += [car.car_plate_number]
        return available_cars

    def book_a_car(self, pickup_time: datetime, drop_off_time: datetime, customer_license_number: str, car_plate_number: str):
        try:
            car = self.inventory.get_car(car_plate_number)
            if car.is_available(pickup_time, drop_off_time):
                booking = Booking(pickup_time, drop_off_time, customer_license_number)
                car.bookings.append(booking)
                self.inventory.update_car(car)
                return f'Your booking is done Pickup: {booking.pickup_time} Dropoff: {booking.drop_off_time} License no: {car_plate_number}'
            else:
                return f"car {car_plate_number} not available for Pickup: {pickup_time} Dropoff: {drop_off_time}"
        except Exception as e:
            print(f"An exception occurred: {str(e)}")


    def return_a_car(self, car_plate_number: str, customer_license_number: str, drop_off_time: datetime):
        try:
            car = self.inventory.get_car(car_plate_number)
            bill = Bill(customer_license_number, car, drop_off_time)
            return f"You have been charged {bill.price} for booking {bill.car_plate_number}"
        except Exception as e:
            print(f"An exception occurred: {str(e)}")

In [2]:
rate = Rate(14,52,200)
car2 = Car(rate, '7853A')
car1 = Car(rate, '7564')
cars = {'7853A': car2, '7564': car1}
inventory = Inventory(cars)
customer = [78653]
inventory.get_all_cars()
inventory.get_car('7853A')
booking_system = BookingCarSystem(inventory)

pickuptime = datetime(2023,10,29)
dropoff = datetime(2023,10,31)

booking_system.show_available_cars(pickuptime, dropoff)

['7853A', '7564']

In [3]:
booking_system.book_a_car(pickuptime, dropoff, 78653, '7564')

'Your booking is done Pickup: 2023-10-29 00:00:00 Dropoff: 2023-10-31 00:00:00 License no: 7564'

In [4]:
dropoff_time = datetime(2023,10,31)

booking_system.return_a_car('7564', 78653, dropoff_time)

'You have been charged 104 for booking 7564'